### Inicio

In [40]:
from datetime import date
import os
os.environ["AWS_PROFILE"] = "men"
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [41]:
params = {
    "class_name":"Números",
    "target_audience_edu_level":"Preescolar",
    "grade":"no aplica"
}

### Retriever

In [42]:
from langchain_aws.retrievers.bedrock import AmazonKnowledgeBasesRetriever

In [43]:
kb_context_id = "BC3JS4SRQG"
kb_result_id = "MQPBML7IQ2"

# Base knowledge
retriever_context = AmazonKnowledgeBasesRetriever(
    knowledge_base_id = kb_context_id,
    retrieval_config = {
        "vectorSearchConfiguration": {
            "numberOfResults": 4,
            'overrideSearchType': "SEMANTIC" # optional
        }
    },
)

retriever_result = AmazonKnowledgeBasesRetriever(
    knowledge_base_id = kb_result_id,
    retrieval_config = {
        "vectorSearchConfiguration": {
            "numberOfResults": 4,
            'overrideSearchType': "SEMANTIC" # optional
        }
    },
)

In [44]:
def get_retriver_docs(docs):    
    context = "\n".join([doc.page_content for doc in docs])  # Iteramos de forma síncrona si 'docs' es una lista
    return context

In [45]:
async def get_retriver_context(params):
    docs = await retriever_context.ainvoke(params)
    return get_retriver_docs(docs)

In [46]:
async def get_retriver_result(params):
    docs = await retriever_result.ainvoke(params)
    return get_retriver_docs(docs)

### Prompts

In [47]:
from langchain.prompts import PromptTemplate

In [48]:
def get_relevant_info():
    with open("1_get_relevant_info.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()
    
    return PromptTemplate(
        input_variables=["class_name"],
        template=prompt_content,
    )



In [49]:
def get_base_prompt():
    with open("2_get_base_prompt.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()

    return PromptTemplate(
        input_variables=["target_audience_edu_level", "class_name", "grade", "date", "context", "learning_result"],
        template=prompt_content
    )

In [50]:
def get_preschool_able_prompt():
    with open("3_get_preschool_able_prompt.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()
    
    return PromptTemplate(
            input_variables=["result_1"],
            template=prompt_content
        )

In [51]:
def get_key_concepts():
    with open("4_get_key_concepts.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()
    
    return PromptTemplate(
            input_variables=["result_1", "target_audience_edu_level", "class_name", "grade", "context"],
            template=prompt_content
        )

In [52]:
def get_activities_prompt():
    with open("5_get_activities_prompt.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()
    
    return PromptTemplate(
            input_variables=["result_1_2", "target_audience_edu_level", "class_name", "grade", "context", "dba"],
            template=prompt_content
        )

In [53]:
def get_exam_prompt():
    with open("6_get_exam_prompt.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()
    
    return PromptTemplate(
            input_variables=["prompt"],
            template=prompt_content
        )

### Definir el LLM


In [54]:
from langchain_aws import BedrockLLM
from langchain_aws import ChatBedrock, ChatBedrockConverse

bedrock_model = "anthropic.claude-3-sonnet-20240229-v1:0"

llm_instance = ChatBedrockConverse(
    model=bedrock_model,
    temperature=0.02,
    max_tokens=None
)

### Parser y Filtro de Relevancia

In [55]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [56]:
relevance = get_relevant_info() | llm_instance | parser
relevance_params = {
    "class_name": params["class_name"]
}
relevante = relevance.invoke(relevance_params)


### Proceso Parte 1 :  Resultados de Aprendizaje

In [57]:
if relevante == 'NO':
  print("El tema que estás buscando no corresponde a un tema formativo, o actualmente no contamos con suficiente información para generar una clase sobre este asunto.")

else:
  prompt_template = get_base_prompt()
  classes = params["class_name"]
  context_query = f"{params['class_name']}, para grados {params['grade']}"
  context = get_retriver_context(context_query)

  dba_query = f"De los derechos basicos de aprendizaje dame la metodologia de enseñanza para la tematica de ({classes}) dirigida a estudiantes del grado {params['grade']}"
  dba = get_retriver_result(dba_query)

  learning_result_query = f"Dame información de los Estándares Básicos de Competencias y Orientaciones pedagógicas para la clase ({classes}) dirigida a estudiantes del grado {params['grade']}"
  learning_result = get_retriver_result(learning_result_query)

  chain = prompt_template | llm_instance | parser

  new_params = {
            "class_name": params["class_name"],
                "target_audience_edu_level":params["target_audience_edu_level"],
                "grade": params["grade"],
                "context": context,
                "dba": dba,
                "learning_result": learning_result,
                "date":  date.today()
            } 
  
  result_1 = chain.invoke(new_params)

  if params["target_audience_edu_level"] == "Preescolar":
      preschool_able = get_preschool_able_prompt() | llm_instance | parser
      preschool_able_params = {"result": result_1}
      result_1 = preschool_able.invoke(preschool_able_params)

C:\Users\Mateo Grisales\AppData\Local\Temp\ipykernel_16380\1491747051.py:18: RuntimeWarning: coroutine 'get_retriver_result' was never awaited
  new_params = {


### Proceso parte 2 : Conceptos clave

In [58]:
key_template = get_key_concepts()

chain_key_concepts = key_template | llm_instance | parser
key_params = {
    "result_1": result_1,
    "class_name": params["class_name"],
    "target_audience_edu_level":params["target_audience_edu_level"],
    "grade": params["grade"],
    "context": context
}

result_2 = chain_key_concepts.invoke(key_params)

if params["target_audience_edu_level"] == "Preescolar":
    result_2 = preschool_able.invoke({"result": result_2})

### Proceso parte 3 : Actividades de aprendizaje

In [59]:
result_1_2 = result_1 + "\n\n" + result_2
activities_template = get_activities_prompt()
chain_activities = activities_template | llm_instance | parser

activities_params = {
    "result_1_2": result_1_2,
    "class_name": params["class_name"],
    "target_audience_edu_level": params["target_audience_edu_level"],
    "grade": params["grade"],
    "context": context,
    "dba": dba
}

result_3 = chain_activities.invoke(activities_params)

if params["target_audience_edu_level"] == "Preescolar":
    result_3 = preschool_able.invoke({"result": result_3})

C:\Users\Mateo Grisales\AppData\Local\Temp\ipykernel_16380\1747868058.py:5: RuntimeWarning: coroutine 'get_retriver_context' was never awaited
  activities_params = {
C:\Users\Mateo Grisales\AppData\Local\Temp\ipykernel_16380\1747868058.py:5: RuntimeWarning: coroutine 'get_retriver_result' was never awaited
  activities_params = {


### Proceso parte 4 : Examen

In [60]:
exam_template = get_exam_prompt()
chain_exam = exam_template | llm_instance | parser
examn_params = {
    "prompt": result_1 + '\n\n' + result_2 + '\n\n' + result_3 
}

result_4 = chain_exam.invoke(examn_params)

if params["target_audience_edu_level"] == "Preescolar":
    result_4 = preschool_able.invoke({"result": result_4})

# Ouput

In [61]:
print(result_1)
print()
print(result_2)
print()
print(result_3)
print()
print(result_4)

# Explorando el Mundo de los Números

Fecha de creación: 2025-01-23
Nivel educativo: Preescolar

## Resultados de aprendizaje:

1. Aprender los nombres de los números del 1 al 10 a través de canciones y juegos.
2. Contar objetos familiares y relacionarlos con los números correspondientes.
3. Dibujar y colorear los números del 1 al 10 siguiendo modelos.
4. Contar objetos en orden y seguir secuencias numéricas simples.
5. Jugar con bloques o fichas para sumar y restar pequeñas cantidades.

## Conceptos clave:

- Juntar cosas
- Cuántas hay
- Contar con los deditos
- Hacer dibujos bonitos
- Conocer los numeritos
- Poner en orden
- Quitar cosas
- Seguir la secuencia

**Tareas específicas:**
1. Juguemos a juntar bloques de colores, ¿cuántos hay en total?
2. Hagamos una manualidad con plastilina, formando bolitas y contándolas.
3. Dibujemos animalitos y contemos cuántas patitas tienen.
4. Cantemos la canción de los números mientras señalamos los deditos.
5. Ordenemos los juguetes por tamaño, 

**Asunto:** Actualizaciones técnicas en el pipeline para mejorar precisión, coherencia y relevancia educativa  

Estimados/as [Nombre del destinatario o equipo]:  

Les informamos sobre las recientes actualizaciones técnicas implementadas en el pipeline de generación de contenido educativo, que abordan problemas de alucinaciones, coherencia en las salidas y ajuste al nivel educativo. Estas mejoras combinan estrategias avanzadas de *prompt engineering* y ajustes en la lógica del código para optimizar los resultados generados.  

### Cambios principales:  

1. **Mitigación de alucinaciones mediante *prompt engineering***:  
   Se diseñó un *prompt* especializado para validar la relevancia y adecuación del tema solicitado antes de proceder con la generación de contenido. Este *prompt* realiza un análisis semántico en cuatro dimensiones clave:  
   - Verifica si el tema tiene valor educativo y es adecuado para un entorno académico.  
   - Descarta automáticamente temas ofensivos, vulgares o inapropiados.  
   - Evalúa si el tema tiene suficiente información o contexto para ser desarrollado.  
   - En caso de nombres de personas, valida su relevancia educativa, histórica o cultural, descartando aquellos que no cumplan estos criterios.  
   Este ajuste garantiza que el sistema minimice alucinaciones al enfocarse exclusivamente en temas relevantes y desarrollables.  

2. **Incremento en la coherencia mediante generación en cascada**:  
   Se reestructuró la lógica de generación para que cada sección de contenido tome como entrada las salidas generadas previamente, logrando mayor cohesión narrativa. Este enfoque evita la independencia entre secciones, un problema que previamente generaba inconsistencias en el flujo. La generación en cascada asegura que los conceptos clave, actividades y evaluaciones estén interconectados y alineados con el contexto general.  

3. **Optimización del contenido para educación preescolar**:  
   Dado que el nivel preescolar presentaba mayores dificultades en términos de adecuación, desarrollamos un *prompt* específico para este nivel educativo. Este ajuste incluye reglas lingüísticas y pedagógicas adaptadas para asegurar que el contenido sea claro, comprensible y apropiado para estudiantes de preescolar.  

### Implementación técnica:  
- Los ajustes en los *prompts* se integraron como plantillas dinámicas que interactúan con el modelo de lenguaje mediante *pipeline chaining*, combinando recuperación de información contextual, generación de conceptos clave, actividades y evaluaciones.  
- Se implementaron consultas semánticas a bases de conocimiento externas para enriquecer el contexto con metodologías, estándares educativos y derechos básicos de aprendizaje.  
- Se emplearon funciones asíncronas para optimizar la recuperación de datos y acelerar el procesamiento.  

Estas mejoras han permitido un aumento significativo en la precisión y consistencia del contenido generado, minimizando errores contextuales y asegurando que los resultados cumplan con las expectativas pedagógicas y técnicas.  

Quedamos atentos/as a cualquier consulta o comentario respecto a estos ajustes.  

Saludos cordiales,  
[Sus nombres]  
[Sus cargos/posiciones]  
[Sus contactos]  


In [63]:
print("""**Asunto:** Actualizaciones técnicas en el pipeline para mejorar precisión, coherencia y relevancia educativa  

Estimados/as [Nombre del destinatario o equipo]:  

Les informamos sobre las recientes actualizaciones técnicas implementadas en el pipeline de generación de contenido educativo, que abordan problemas de alucinaciones, coherencia en las salidas y ajuste al nivel educativo. Estas mejoras combinan estrategias avanzadas de *prompt engineering* y ajustes en la lógica del código para optimizar los resultados generados.  

### Cambios principales:  

1. **Mitigación de alucinaciones mediante *prompt engineering***:  
   Se diseñó un *prompt* especializado para validar la relevancia y adecuación del tema solicitado antes de proceder con la generación de contenido. Este *prompt* realiza un análisis semántico en cuatro dimensiones clave:  
   - Verifica si el tema tiene valor educativo y es adecuado para un entorno académico.  
   - Descarta automáticamente temas ofensivos, vulgares o inapropiados.  
   - Evalúa si el tema tiene suficiente información o contexto para ser desarrollado.  
   - En caso de nombres de personas, valida su relevancia educativa, histórica o cultural, descartando aquellos que no cumplan estos criterios.  
   Este ajuste garantiza que el sistema minimice alucinaciones al enfocarse exclusivamente en temas relevantes y desarrollables.  

2. **Incremento en la coherencia mediante generación en cascada**:  
   Se reestructuró la lógica de generación para que cada sección de contenido tome como entrada las salidas generadas previamente, logrando mayor cohesión narrativa. Este enfoque evita la independencia entre secciones, un problema que previamente generaba inconsistencias en el flujo. La generación en cascada asegura que los conceptos clave, actividades y evaluaciones estén interconectados y alineados con el contexto general.  

3. **Optimización del contenido para educación preescolar**:  
   Dado que el nivel preescolar presentaba mayores dificultades en términos de adecuación, desarrollamos un *prompt* específico para este nivel educativo. Este ajuste incluye reglas lingüísticas y pedagógicas adaptadas para asegurar que el contenido sea claro, comprensible y apropiado para estudiantes de preescolar.  

### Implementación técnica:  
- Los ajustes en los *prompts* se integraron como plantillas dinámicas que interactúan con el modelo de lenguaje mediante *pipeline chaining*, combinando recuperación de información contextual, generación de conceptos clave, actividades y evaluaciones.  
- Se implementaron consultas semánticas a bases de conocimiento externas para enriquecer el contexto con metodologías, estándares educativos y derechos básicos de aprendizaje.  
- Se emplearon funciones asíncronas para optimizar la recuperación de datos y acelerar el procesamiento.  

Estas mejoras han permitido un aumento significativo en la precisión y consistencia del contenido generado, minimizando errores contextuales y asegurando que los resultados cumplan con las expectativas pedagógicas y técnicas.  

Quedamos atentos/as a cualquier consulta o comentario respecto a estos ajustes.  

Saludos cordiales,  
[Sus nombres]  
[Sus cargos/posiciones]  
[Sus contactos]  
""")

**Asunto:** Actualizaciones técnicas en el pipeline para mejorar precisión, coherencia y relevancia educativa  

Estimados/as [Nombre del destinatario o equipo]:  

Les informamos sobre las recientes actualizaciones técnicas implementadas en el pipeline de generación de contenido educativo, que abordan problemas de alucinaciones, coherencia en las salidas y ajuste al nivel educativo. Estas mejoras combinan estrategias avanzadas de *prompt engineering* y ajustes en la lógica del código para optimizar los resultados generados.  

### Cambios principales:  

1. **Mitigación de alucinaciones mediante *prompt engineering***:  
   Se diseñó un *prompt* especializado para validar la relevancia y adecuación del tema solicitado antes de proceder con la generación de contenido. Este *prompt* realiza un análisis semántico en cuatro dimensiones clave:  
   - Verifica si el tema tiene valor educativo y es adecuado para un entorno académico.  
   - Descarta automáticamente temas ofensivos, vulgares 